In [1]:
#### GRI GENERATOR

In [1]:
### EXTRACTING GRI SOURCE DATA FROM GENERAL MS EXCEL SOURCE
def get_gri_data_from_excel():
    ### Importing standard modules and date-special modules:
    import numpy as np
    import pandas as pd
    from datetime import date
    ### Source xlsx file attributes:
    path_gri_data_xlsx = 'Data_Files/Source_Files/mri_data.xlsx'
    tab_gri_model = 'Model 01'  
    ### Reading Model information from Source model sheet:
    df_model_raw = pd.read_excel(path_gri_data_xlsx, sheet_name = tab_gri_model, header = 1, usecols = [0, 1, 2, 3, 4, 5, 6])
    ### Group border rows deleting:
    df_model_raw = df_model_raw[df_model_raw['Asset Group'] != df_model_raw['Asset Code']]   
    ### Dividing list on asset part and MRI weights part:
    df_model_asset = df_model_raw[df_model_raw['Asset Group'] != 'MRI'] ### Asset part
    df_model_asset.reset_index(drop = True, inplace = True)
    df_model_gri = df_model_raw[df_model_raw['Asset Group'] == 'MRI'] ### MRI part
    df_model_gri.reset_index(drop = True, inplace = True) 
    ### Aggregating data from the source xlsx file to pd.DataFrame:
    arr_tab_data = []
    for iter_index, iter_row in df_model_asset.iterrows():
        iter_tab = iter_row['Asset Tab Name']
        iter_asset = iter_row['Asset Code']
        ser_iter_tab = pd.read_excel(path_gri_data_xlsx, sheet_name = iter_tab, header = 0, index_col = 0, squeeze = True)
        ser_iter_tab.name = iter_asset
        arr_tab_data.append(ser_iter_tab)
    df_source_data = pd.concat(arr_tab_data, axis = 1, join = 'outer')  
    df_source_data.index.name = 'Date'
    df_source_data = df_source_data.astype('float64')

    return [df_model_asset, df_model_gri, df_source_data]
    
### EXTRACTING SOURCE DATA FROM MS EXCEL FILES AND SAVING TO HDF FILES SCRIPT
### Importing standard modules and date-special modules:
import numpy as np
import pandas as pd
from datetime import date
### Global parameters declaring:
date_first = date(1990, 1, 1)
date_last = date(2018, 12, 31)
### Declaring constants:
path_gri_data_hdf = 'Data_Files/Source_Files/gri_data.h5'
model_asset_key = 'model_asset_key'
model_gri_key = 'model_gri_key'
source_data_key = 'source_data_key'
### Data extracting:
[df_model_asset, df_model_gri, df_source_data] = get_gri_data_from_excel()
### closing files that have been previously opened:
import tables
tables.file._open_files.close_all()
### Saving data to hdf5 table formatted files:
df_model_asset.to_hdf(path_gri_data_hdf, model_asset_key, mode = 'w', format = 'fixed')
df_model_gri.to_hdf(path_gri_data_hdf, model_gri_key, mode = 'a', format = 'fixed')
df_source_data.to_hdf(path_gri_data_hdf, source_data_key, mode = 'a', format = 'fixed')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['Asset Group', 'Asset Group Description', 'Asset Code', 'Asset Tab Name', 'Asset Desc', 'Processing Routine']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [2]:
### TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING
### Importing standard modules and date-special modules:
import numpy as np
import pandas as pd
from datetime import date
### Declaring constants:
path_gri_data_hdf = 'Data_Files/Source_Files/gri_data.h5'
model_asset_key = 'model_asset_key'
model_gri_key = 'model_gri_key'
source_data_key = 'source_data_key'

df_model_asset = pd.read_hdf(path_gri_data_hdf, model_asset_key)
df_model_gri = pd.read_hdf(path_gri_data_hdf, model_gri_key)
df_source_data = pd.read_hdf(path_gri_data_hdf, source_data_key)

In [4]:
def get_rolling_z_score(ser_source, min_wnd, max_wnd, winsor_bottom, winsor_top):
    ### Importing standard modules:    
    import numpy as np
    import pandas as pd
    
    ### Initializing z-score vector:
    ser_asset_z_score = pd.Series(np.NaN, index = ser_source.index)
    ser_asset_z_score = ser_asset_z_score.astype('float64')
    if (ser_source.count() > 0):
        ### Initialising index conditions:
        asset_start_index = ser_source.first_valid_index() + pd.offsets.BusinessDay(min_wnd - 1)  
        iter_date_index = ser_source.index[-1]
        ### Checking for at list min_wnd elements of rolling window are not np.NaN:
        if (iter_date_index >= asset_start_index):          
            ### Isolating rolling window for particular data vector element:
            iter_start_index = iter_date_index - pd.offsets.BusinessDay(max_wnd)
            ser_iter_source = ser_source.loc[iter_start_index : iter_date_index]        
            ser_iter_z_score = (ser_iter_source - ser_iter_source.mean()) / ser_iter_source.std()            
            ### Winsorization process:
            bool_to_winsor = True            
            while (bool_to_winsor): 
                ### Value based winsorization:                
                ser_iter_z_score.clip(lower = winsor_bottom, upper = winsor_top, inplace = True)
                ### Recalculating of z scores:
                ser_iter_z_score = (ser_iter_z_score - ser_iter_z_score.mean()) / ser_iter_z_score.std()
                ### Checking for boundaries:
                if (ser_iter_z_score[(ser_iter_z_score <= (winsor_bottom - 0.01)) | (ser_iter_z_score >= (winsor_top + 0.01))].count() == 0):
                    bool_to_winsor = False
            ### Filling z vector part after the winsorizing (if needed):
            ser_asset_z_score.loc[iter_start_index : iter_date_index] = ser_iter_z_score.values
    return ser_asset_z_score

In [5]:
    ### TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING
    ### Importing standard modules and date-special modules:    
    import numpy as np
    import pandas as pd
    from datetime import date    
    ### Declaring constants:
    date_start = date(1993, 12, 31)    
    path_gri_data_hdf = 'Data_Files/Source_Files/gri_data.h5'
    model_asset_key = 'model_asset_key'
    model_gri_key = 'model_gri_key'
    source_data_key = 'source_data_key'
    ### Limitations for rolling windows for z-score calculating:
    asset_window_min = 252
    asset_window_max = 252 * 100
    ### Limitations for z-score winsorizing:
    arr_winsor_boundary = [-4, 4]
    asset_code = 'iv_rvx'
    ser_test = get_rolling_z_score(np.log(df_source_data[asset_code]), asset_window_min, asset_window_max, arr_winsor_boundary[0], arr_winsor_boundary[1])

In [5]:
def get_gri_value(iter_date):
    ### Importing standard modules and date-special modules:    
    import numpy as np
    import pandas as pd
    from datetime import date    
    ### Declaring constants:
    date_first = date(1990, 1, 1)    
    date_start = date(1993, 12, 31)    
    path_gri_data_hdf = 'Data_Files/Source_Files/gri_data.h5'
    model_asset_key = 'model_asset_key'
    model_gri_key = 'model_gri_key'
    source_data_key = 'source_data_key'
    ### Limitations for rolling windows for z-score calculating:
    asset_window_min = 252
    asset_window_max = 252 * 100
    mri_window_max = 260 * 10
    ### Limitations for z-score winsorizing:
    winsor_bottom = -4
    winsor_top = 4    
    ### Limitations for moving average for GRI calculation:
    ma_max_wnd = 5    
    ### Extracting model data from hdf5 file:
    df_model_asset = pd.read_hdf(path_gri_data_hdf, model_asset_key)
    df_model_gri = pd.read_hdf(path_gri_data_hdf, model_gri_key)
    df_source_data = pd.read_hdf(path_gri_data_hdf, source_data_key).loc[ : iter_date]
    df_source_data.fillna(method = 'ffill', inplace = True)
    index_selected = pd.date_range(date_first, iter_date, freq = 'B')
    df_selected_data = df_source_data.reindex(index_selected, method = 'ffill')    
    ### Base assets determination (resorting by earliest value):
    df_model_asset['Asset Date'] = date(1900, 1, 1)
    for (iter_index, asset_code) in df_model_asset['Asset Code'].iteritems():
        df_model_asset.loc[iter_index, 'Asset Date'] = df_selected_data[asset_code].dropna().index.min() 
    df_model_asset.sort_values(['Asset Group', 'Asset Date'], inplace = True)
    df_model_asset = df_model_asset.reset_index(drop = True)    
    ### Initialising loop visibility variables:          
    dict_group_container = {} ### Group vectors collection for gri mean vector calculation   
    ### Standartizing loop on group level:
    for asset_group_name, df_asset_group in df_model_asset.groupby('Asset Group'):
        ### Initialising group visibility variables:
        bool_base_asset = True
        dict_asset_container = {} ### Asset vectors collection for group mean matrix calculation
        ### Standartizing cycle on asset level with the group:
        for (asset_index, asset_code) in df_asset_group['Asset Code'].iteritems():
            ### Assignment of base asset data set:
            if (bool_base_asset):
                bool_base_asset = False
                ### Performing z scoring for base asset:
                ser_base_source = np.log(df_selected_data[asset_code][: iter_date])
                ser_base_z_score = get_rolling_z_score(ser_base_source, 
                                                       asset_window_min, asset_window_max,
                                                       winsor_bottom, winsor_top)
                ### Calculating ethalon filled quantity before date_start:
                int_base_filled = ser_base_source[ : date_start].count()
                ### Adding result to dictionary:
                dict_asset_container[asset_code] = ser_base_z_score
            ### Normalization of other asset's data sets:                
            else:
                ### Performing z scoring for asset:        
                ser_asset_source = np.log(df_selected_data[asset_code][: iter_date])                
                ser_asset_z_score = get_rolling_z_score(ser_asset_source, 
                                                        asset_window_min, asset_window_max,
                                                        winsor_bottom, winsor_top)
                ### Calculating asset filled quantity:                
                int_asset_filled = ser_asset_source[ : date_start].count()          
                ### Standartizing asset if it does not have enough initial values:
                if (int_asset_filled < int_base_filled * 2 / 3):
                    ### RenormaLizing asset z vector with base z vector data:
                    if (ser_asset_z_score.first_valid_index() != None):                
                        index_asset_start = ser_asset_z_score.first_valid_index() + pd.offsets.BusinessDay(asset_window_min - 1)
                        ser_base_z_part = ser_base_z_score.loc[index_asset_start - pd.offsets.BusinessDay(asset_window_min - 1) : iter_date]    
                        ser_asset_z_score = ser_asset_z_score * ser_base_z_part.std() + ser_base_z_part.mean()
                ### Adding result to dictionary:  
                dict_asset_container[asset_code] = ser_asset_z_score   
        ### Calculating z vector for group:
        ser_group_mean = pd.concat(dict_asset_container, axis = 0, names = ['Asset Code', 'Date'], copy = False)
        ser_group_mean = ser_group_mean.groupby('Date').mean()    
        ser_group_mean_z = (ser_group_mean - ser_group_mean.mean()) / ser_group_mean.std()  
        dict_group_container[asset_group_name] = ser_group_mean_z
    ### Model groups mean calculation:    
    ser_model_mean = pd.concat(dict_group_container, axis = 0, names = ['Group', 'Date'], copy = False)
    ser_model_mean = ser_model_mean.groupby(['Date']).mean()  
    ### GRI z-scoring:
    ser_gri_z_score = pd.Series(np.NaN, index = ser_model_mean.index)
    if (iter_date >= pd.Timestamp(date_start)):
        ser_iter_mri = ser_model_mean.loc[iter_date - pd.offsets.BusinessDay(mri_window_max) : iter_date]
        ser_iter_z_score = (ser_iter_mri - ser_iter_mri.mean()) / ser_iter_mri.std()
        ### Winsorization process:
        bool_to_winsor = True            
        while (bool_to_winsor):       
            ### Value based winsorization:
            ser_iter_z_score.clip(lower = winsor_bottom, upper = winsor_top, inplace = True)
            ### Recalculating of z scores:
            ser_iter_z_score = (ser_iter_z_score - ser_iter_z_score.mean()) / ser_iter_z_score.std()                
            ### Checking for boundaries:
            if (ser_iter_z_score[(ser_iter_z_score <= (winsor_bottom - 0.01)) | (ser_iter_z_score >= (winsor_top + 0.01))].count() == 0):                    
                bool_to_winsor = False    
        ser_gri_z_score.loc[iter_date - pd.offsets.BusinessDay(mri_window_max) : iter_date] = ser_iter_z_score.values
    ### Calculating z matrix for MRI with winsorization:       
    ser_gri_z_ma = ser_gri_z_score.copy()
    for iter_shift in np.arange(1, ma_max_wnd):
        ser_gri_z_ma = ser_gri_z_ma + ser_gri_z_score.shift(iter_shift)
    ser_gri_z_ma = ser_gri_z_ma / ma_max_wnd
    
    return ser_gri_z_ma

In [6]:
### LOOPER FOR GRI
### Importing standard modules and date-special modules:
import numpy as np
import pandas as pd
from datetime import date
### Global parameters declaring:
date_start = date(1993, 12, 31)  
date_last = date(2018, 12, 31)
date_range_test = pd.date_range(date_start, date_last, freq = 'B')
#date_test = date(2018, 1, 1)
#date_range_test = pd.date_range(date_test, date_last, freq = 'B')
dict_gri_vector = {}
iter_counter = 0
for iter_date in date_range_test:
    iter_counter = iter_counter + 1
    dict_gri_vector[iter_date] = get_gri_value(iter_date)
    if ((iter_counter // 1000) == (iter_counter / 1000)):
        print('Progress printout',iter_counter, '/', iter_date)
ser_gri_full = pd.concat(dict_gri_vector)
ser_gri_released = pd.Series(np.NaN, index = ser_gri_full.index.get_level_values(0).unique())
for iter_date in ser_gri_released.index:
    ser_gri_released[iter_date] = ser_gri_full.loc[iter_date, iter_date]
ser_gri_released.index.name = 'Date'
ser_gri_released.name = 'GRI_MA5'

Progress printout 1000 / 1997-10-30 00:00:00
Progress printout 2000 / 2001-08-30 00:00:00
Progress printout 3000 / 2005-06-30 00:00:00
Progress printout 4000 / 2009-04-30 00:00:00
Progress printout 5000 / 2013-02-28 00:00:00
Progress printout 6000 / 2016-12-29 00:00:00


In [7]:
### RESULTS SAVING
path_gri_index_hdf = 'Data_Files/Source_Files/gri_released_index.h5'
gri_vector_key = 'gri_vector_key'
gri_vintage_key = 'gri_vintage_key'
gri_start_key = 'gri_start_key'
ser_gri_released.to_hdf(path_gri_index_hdf, gri_vector_key, mode = 'w', format = 'fixed')
ser_gri_full.index.names = ['Date_Point', 'Date']
ser_gri_full.to_hdf(path_gri_index_hdf, gri_vintage_key, mode = 'a', format = 'fixed')
ser_gri_full.loc[date_start, :].to_hdf(path_gri_index_hdf, gri_start_key, mode = 'a', format = 'fixed')

In [9]:
### TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING
import numpy as np
import pandas as pd
from datetime import date
### Global parameters declaring:
date_first = date(1990, 1, 1)
date_last = date(2018, 12, 31)
date_range_test = pd.date_range(date_first, date_last, freq = 'B')
path_mri_index_hdf = 'Data_Files/Source_Files/gri_released_index.h5'
object_diag_mri_hdf = 'diag_MRI_data'
object_released_mri_hdf = 'released_MRI_data'
ser_mri_z_diag = pd.read_hdf(path_mri_index_hdf, key = object_diag_mri_hdf)
ser_mri_released = pd.read_hdf(path_mri_index_hdf, key = object_released_mri_hdf)
path_gri_index_hdf = 'Data_Files/Source_Files/gri_released_index.h5'
gri_vector_key = 'gri_vector_key'
gri_vintage_key = 'gri_vintage_key'
ser_gri_released = pd.read_hdf(path_gri_index_hdf, gri_vector_key)
print('GRI/MRI delta sum:', abs(ser_mri_released[date_range_test] - ser_gri_released).sum())
print('GRI/MRI delta min:', abs(ser_mri_released[date_range_test] - ser_gri_released).min())
print('GRI/MRI delta mean:', abs(ser_mri_released[date_range_test] - ser_gri_released).mean())
print('GRI/MRI delta max:', abs(ser_mri_released[date_range_test] - ser_gri_released).max())
print('GRI/MRI delta std:', abs(ser_mri_released[date_range_test] - ser_gri_released).std())
import matplotlib
ser_gri_released.plot(figsize = (15, 5))

GRI/MRI delta sum: 0.0027388925416881884
GRI/MRI delta min: 2.9926450206829713e-11
GRI/MRI delta mean: 4.1994672518984796e-07
GRI/MRI delta max: 5.781425458373235e-06
GRI/MRI delta std: 4.133255916735697e-07
